In [42]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.4f}'.format
plt.style.use('seaborn')

/var/folders/5z/wzj0qt495bbdktpqnnk1nx100000gn/T/ipykernel_6741/3960066781.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
def getYData(symbol):
    start = "2014-10-01"
    end = "2021-05-28"
    return yf.download(symbol,start,end)

In [3]:
msft = getYData('msft')

[*********************100%***********************]  1 of 1 completed


In [5]:
msft

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-10-01,46.2700,46.5300,45.8500,45.9000,39.8098,38088400
2014-10-02,45.8300,46.1000,45.6400,45.7600,39.6883,25119400
2014-10-03,45.9800,46.3000,45.6100,46.0900,39.9745,32453200
2014-10-06,46.1200,46.3000,45.9200,46.0900,39.9745,20604000
2014-10-07,45.8600,45.9300,45.4200,45.5300,39.4888,25723700
...,...,...,...,...,...,...
2021-05-21,247.5700,248.3300,244.7400,245.1700,241.9872,21863100
2021-05-24,247.7900,251.1600,247.5100,250.7800,247.5244,21411500
2021-05-25,251.7700,252.7500,250.8200,251.7200,248.4522,17704300


In [6]:
msft.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True)
msft.rename(columns={'Close':'Price'},inplace=True)
msft

,Price
Date,
2014-10-01,45.9000
2014-10-02,45.7600
2014-10-03,46.0900
2014-10-06,46.0900
2014-10-07,45.5300
...,...
2021-05-21,245.1700
2021-05-24,250.7800
2021-05-25,251.7200


In [19]:
msft['lag'] = msft.Price.shift(periods=1)
msft['diff1'] = msft.Price.sub(msft.lag)
msft['diff2'] = msft.Price.diff(periods=1)
msft['Returns'] = msft.Price.pct_change(periods=1)

In [20]:
msft.drop(columns=['diff1','diff2', 'lag'],inplace=True)

In [22]:
msft

,Price,Returns
Date,,
2014-10-01,45.9000,NaN
2014-10-02,45.7600,-0.0031
2014-10-03,46.0900,0.0072
2014-10-06,46.0900,0.0000
2014-10-07,45.5300,-0.0122
...,...,...
2021-05-21,245.1700,-0.0053
2021-05-24,250.7800,0.0229
2021-05-25,251.7200,0.0037


In [24]:
msft.describe()

,Price,Returns
count,1676.0000,1675.0000
mean,106.2288,0.0012
std,60.6893,0.0173
min,40.2900,-0.1474
25%,54.8225,-0.0062
50%,89.6600,0.0009
75%,138.9000,0.0091
max,261.9700,0.1422


- mean return => average return per day
- std => measure of the risk/volatility 


## Investment multiple and CAGR

In [26]:
multiple = (msft.Price[-1]/msft.Price[0])
multiple

5.4315901801883415

In [28]:
growth = (multiple-1)*100
growth

443.15901801883416

#### CAGR 
- constant annual rate of return that would require an investment to grow from its start to end, assuming profits were reinvested at the end of each year

In [30]:
td_years = ((msft.index[-1]-msft.index[0]).days)/365.25
td_years

6.652977412731006

In [32]:
cagr = multiple**(1/td_years)-1
cagr

0.2896322305109247

## Compund Returns and Geometric Mean Return

In [37]:
multiple = (1+msft.Returns).prod()
n = msft.Returns.count()
multiple

5.43159018018834

#### Geometric mean return is daily return growth rate

In [38]:
geo_mean = multiple**(1/n)-1
geo_mean

0.0010107982390816783

#### Arithmetic mean return is always greater than Geometric mean return

In [39]:
mu = msft.Returns.mean()
mu

0.0011595881894089248

## Discrete Compounding

In [40]:
PV = 100
r = 0.08
n = 1
m = 100000


In [44]:
FV = PV *(1+r/m)**(n*m)
FV

108.32870330122834

In [45]:
msft["log_ret"] = np.log(msft.Price/msft.Price.shift())
msft

,Price,Returns,log_ret
Date,,,
2014-10-01,45.9000,NaN,NaN
2014-10-02,45.7600,-0.0031,-0.0031
2014-10-03,46.0900,0.0072,0.0072
2014-10-06,46.0900,0.0000,0.0000
2014-10-07,45.5300,-0.0122,-0.0122
...,...,...,...
2021-05-21,245.1700,-0.0053,-0.0053
2021-05-24,250.7800,0.0229,0.0226
2021-05-25,251.7200,0.0037,0.0037


In [46]:
trading_days_year = msft.Returns.count()/(((msft.index[-1]-msft.index[0]).days)/365.25)
trading_days_year

251.766975308642

In [47]:
np.exp(msft.log_ret.mean() * trading_days_year)-1

0.28963223051092424